![](https://images.pexels.com/photos/59822/household-cavalry-soldier-mounted-royal-soldier-59822.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260)

# Quaternions

Kevin J. Walchko, Phd

---

Unit quaternions provide a convenient mathematical notation for representing orientations and rotations of objects in three dimensions. Compared to Euler angles they are simpler to compose and avoid the problem of gimbal lock. Compared to rotation matrices they are more compact, more numerically stable, and more efficient.

Quaternions are composed of a real component (w) and an imaginary component (x,y,z) which has 3 elements. Thus quaternions use 4 variables to represent 3 rotational elements (think Euler: roll, pitch, and yaw) which means they have a redundant parameter. This redundancy allows them to avoid issues seen in Euler. The only problem with them is you have to be able to visualize 3D rotations in 4D space ... good luck! Thus, for humans to visualize them, we typically transform them back to Euler angles. However, the issues we see in Euler angles (gimbal lock) can reappear when we transform them. Note, the gimbal lock only is an issue in Euler angles, quaternions are fine, it is just the transformation to Euler that can have an issue.

## References

- [Quaternion Integration](https://www.ashwinnarayan.com/post/how-to-integrate-quaternions/)
- Wikipedia: [Convert Between Quaternions and Euler Angles](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)
- Wikipedia: [Euler Angle Definitions](https://en.wikipedia.org/wiki/Euler_angles#Conventions_2)
- Wikipedia: [Gimbal Lock](https://en.wikipedia.org/wiki/Gimbal_lock)

In [1]:
from squaternion import Quaternion
from math import pi
from math import sqrt, sin, cos, atan2, asin, acos
import numpy as np
from numpy.testing import assert_allclose

### Rigid Bodies Rotations

A rigid body can be rotated angle $\mu$ about an arbitrary moving/fixed axis ($\hat e$) in space by:

$$
q_{x,y,z} = \hat e \sin( \frac{\mu}{2} ) \\
q_r = \cos(\frac{\mu}{2} )
$$

Quaternion multiplication ($\otimes$) is:

$$
q \otimes p =
\begin{bmatrix}
    a & -b & -c & -d \\
    b &  a & -d &  c \\
    c &  d &  a & -b \\
    d & -c &  b &  a \\
\end{bmatrix} \cdot p = Q \cdot p \\
$$

Quaternion differential equation:

$$
\dot q = \frac{1}{2} q \otimes w \\
w = \begin{bmatrix} 0 & \omega_b \end{bmatrix}^T \\
\dot q = \frac{1}{2} W q \\
W =
\begin{bmatrix}
    0   & -w_x & -w_y & -w_z \\
    w_x & 0    & w_z  & -w_y \\
    w_y & -w_z & 0    & w_x \\
    w_z & w_y  & -w_x & 0
\end{bmatrix}
$$

For discrete timeframes:

$$
q_{k+1} = exp(\frac{1}{2} w \Delta T) \otimes q_k \\
q_{k+1} = q_k + (\frac{1}{2} w \Delta T) \otimes q_k 
$$

# Simple Quaternions

Generally I don't need all of the capabilities (or complexity) of quaternion math libraries. Basically I just need a way to convert between Euler and Quaternion representations and have a nice way to print them out.

[squaterion](https://pypi.org/project/squaternion/) is a quaterion library. You can install it with:

```
pip3 install squaternion
```

In [2]:
# create a simple quaterion with no rotation
q = Quaternion.from_euler(0,0,0)
w,x,y,z = q
print(q)
print(w, x, y, z)

Quaternion(w=1.0, x=0.0, y=0.0, z=0.0)
1.0 0.0 0.0 0.0


In [3]:
# let's make up some euler angles (roll, pitch, yaw)
euler = (90,45,180,)
print(f"Euler: {euler} deg")

# let's create a quaternion
print('-'*40)
q = Quaternion.from_euler(*euler, degrees=True)
print(q)

# let's double check euler => quaternion => euler worked
e = q.to_euler(degrees=True)

print('-'*40)
print(f"Euler: {e} deg")

# if you look at the answers they are the same except for some small rounding errors

Euler: (90, 45, 180) deg
----------------------------------------
Quaternion(w=0.2705980500730985, x=-0.27059805007309845, y=0.6532814824381882, z=0.6532814824381883)
----------------------------------------
Euler: (89.99999999999997, 44.99999999999999, 180.0) deg


# Test

![](https://upload.wikimedia.org/wikipedia/en/thumb/3/30/Plane_with_ENU_embedded_axes.svg/425px-Plane_with_ENU_embedded_axes.svg.png)

So let's do a simple test of iterating over a bunch of Euler angles, convert them to a quaternion and then back to Euler and see if we get the same answer.

    euler => quaternion => euler

Now, Euler angles have a sinularity around the following locations:

- Roll:  [-$\pi$, $\pi$]
- Pitch: [-$\pi$/2, $\pi$/2]
- Yaw:   [-$\pi$, $\pi$]

Anything outside of these Euler angle ranges will not work, unless you take special percausions. For my applications, this isn't an issue.

So let's run through a range of valid Euler angles and do the transforms and see if we have an issue. Note, because of small rounding errors, we check if the answers are the same within 0.001 degrees. If **no errors print out** and all you see is **Done**, then everything went well.

In [4]:
# run though and make sure there are no errors
# https://en.wikipedia.org/wiki/Euler_angles#Conventions_2
# valid ranges:
# asin: [-pi/2, pi/2]
# cos: [0, pi]
# atan2: [-pi,pi]
#---------------------
# valid euler angles, meaning no gimbal lock issues
# roll: [-pi,pi]
# pitch: [-pi/2, pi/2]
# yaw: [-pi,pi]
delta = 10
for i in range(-179, 180, delta):
    for j in range(-89, 90, delta):
        for k in range(-179,180, delta):
            q = Quaternion.from_euler(i,j,k, degrees=True)  # euler => quat
            e = q.to_euler(degrees=True)     # quat => euler
            for a, b in zip((i,j,k,), e):
                if abs(a - b) > 0.001:  # are the answers within 0.001 degrees?
                    print('-'*40)
                    print('Error')
                    print(i,j,k, '==', e)
                    print(q)
print("Done")

Done


# Misc

In [5]:
# https://www.mathworks.com/help/fusion/ref/quaternion.rotmat.html
# my stuff follows ZYX frame not point
q = Quaternion.from_euler(30,45,0,True)
print(q) # 0.8924 +  0.23912i +  0.36964j - 0.099046k

print(q.angle*180/pi)
print(q.axis)

Quaternion(w=0.8923991008325228, x=0.23911761839433449, y=0.3696438106143861, z=-0.09904576054128762)
53.64743527556287
(0.5299040755263686, 0.8191607253909541, -0.21949345483979882)


In [6]:
# point
#  0.7071   -0.0000    0.7071
#  0.3536    0.8660   -0.3536
# -0.6124    0.5000    0.6124
#
# frame [works]
# 0.7071   -0.0000   -0.7071
# 0.3536    0.8660    0.3536
# 0.6124   -0.5000    0.6124
a,b,c,d = q
# r = np.array([ # point [broke]
#     [2*a**2-1+2*b**2, 2*b*c-2*a*d, 2*b*d+2*a*c],
#     [2*b*c+2*a*d, 2*a**2-1+2*c**2, 2*c*d-2*a*b],
#     [2*b*d-2*a*c, 2*c*d+2*a*b, 2*a**2-1+2*d**2]
# ])
r = np.array([ # frame [works]
    [2*a**2-1+2*b**2, 2*b*c+2*a*d, 2*b*d-2*a*c],
    [2*b*c-2*a*d, 2*a**2-1+2*c**2, 2*c*d+2*a*b],
    [2*b*d+2*a*c, 2*c*d-2*a*b, 2*a**2-1+2*d**2]
])
print(r)

[[ 7.07106781e-01 -2.77555756e-17 -7.07106781e-01]
 [ 3.53553391e-01  8.66025404e-01  3.53553391e-01]
 [ 6.12372436e-01 -5.00000000e-01  6.12372436e-01]]


In [7]:
from rotations.rotations import R1,R2

In [8]:
rr = R2(45,True).dot(R1(30,True))
print(rr)

[[ 0.70710678  0.35355339 -0.61237244]
 [ 0.          0.8660254   0.5       ]
 [ 0.70710678 -0.35355339  0.61237244]]


In [9]:
v = np.array([1,2,3])
qc=np.array(q.conjugate)
qq=np.array(q)
vq=np.array([0,*v])

In [10]:
s=np.outer(qc,vq)
print(s)
x=s.dot(qq)
print(x)
x[1:]/x[0]

[[ 0.          0.8923991   1.7847982   2.6771973 ]
 [-0.         -0.23911762 -0.47823524 -0.71735286]
 [-0.         -0.36964381 -0.73928762 -1.10893143]
 [ 0.          0.09904576  0.19809152  0.29713728]]
[ 0.60796291 -0.16290317 -0.25182648  0.0674767 ]


array([-0.26794919, -0.41421356,  0.11098819])

In [11]:
# matlab seems to the transpose of wikipedia???
a,b,c,d = q
r = np.array([ # frame [works]
    [2*a**2-1+2*b**2, 2*b*c+2*a*d, 2*b*d-2*a*c],
    [2*b*c-2*a*d, 2*a**2-1+2*c**2, 2*c*d+2*a*b],
    [2*b*d+2*a*c, 2*c*d-2*a*b, 2*a**2-1+2*d**2]
])
print(r)

r.dot(v)

[[ 7.07106781e-01 -2.77555756e-17 -7.07106781e-01]
 [ 3.53553391e-01  8.66025404e-01  3.53553391e-01]
 [ 6.12372436e-01 -5.00000000e-01  6.12372436e-01]]


array([-1.41421356,  3.14626437,  1.44948974])

In [12]:
q.to_rot()

((0.7071067811865476, 0.35355339059327373, 0.6123724356957946),
 (-2.7755575615628914e-17, 0.8660254037844387, -0.49999999999999994),
 (-0.7071067811865476, 0.3535533905932737, 0.6123724356957946))

In [15]:
np.allclose(np.array(q.to_rot()).T, r) # all good rot.T == rot ???

True

In [16]:
np.allclose(np.array(q.to_rot()), r) # fail

False